# Capstone Project

### Introduction
#### This notebook is part of the capstone project of coursera's course called applied Data Science Capstone

### Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Getting and structuring the data</a>

2. <a href="#item2">Exploring the dataset</a>

3. <a href="#item3">Analyzing the data</a>

</font>
</div>

In [1]:
#Libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from folium.features import DivIcon

### 1. Getting  and structuring the data

#### Getting Toronto Data

In [2]:
# Getting data about Toronto Neighborhoods and their respective latitudes and logintudes
filename = "Toronto_Neighborhoods_locations.csv"

df = pd.read_csv(filename)

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Droping rows that Borough != from "Toronto"

In [3]:
TorontoDf = df.drop(df[~df["Borough"].str.contains("Toronto", na=False)].index).reset_index(drop=True)

#### Rename column "Neighbourhood" to "Neighborhood"

In [4]:
TorontoDf.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [5]:
TorontoDf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Getting Toronto latitude and logintude

In [6]:
# Getting logintude and latitude of Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 2. Exploring the dataset

####  Creating a map of Toronto's neighborhoods

In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to m
for lat, lng, borough, neighborhood in zip(TorontoDf['Latitude'], TorontoDf['Longitude'], TorontoDf['Borough'], TorontoDf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto



#### Getting data from Foursquare

In [8]:
#Credentials
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Getting best restaurants from Toronto

In [9]:
LIMIT = 20 # limit of number of venues returned by Foursquare API

query = "Good Restaurants"

near = "Toronto, ON, Canadá"

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&query={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    near,
    query,
    LIMIT)

In [10]:
# Send the request
best_restaurants = requests.get(url).json()

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
restaurants = best_restaurants['response']['groups'][0]['items']
    
best_restaurants = json_normalize(restaurants) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
best_restaurants = best_restaurants.loc[:, filtered_columns]

# filter the category for each row
best_restaurants['venue.categories'] = best_restaurants.apply(get_category_type, axis=1)

# clean columns
best_restaurants.columns = [col.split(".")[-1] for col in best_restaurants.columns]

best_restaurants.head()

<ipython-input-12-d367e13234f0>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  best_restaurants = json_normalize(restaurants) # flatten JSON


,id,name,categories,lat,lng
0,4ad4c05ef964a52096f620e3,CN Tower,Monument / Landmark,43.642536,-79.387182
1,4b49183ff964a520a46526e3,Terroni,Italian Restaurant,43.650927,-79.375602
2,4be8b576d837c9b6e2b6a506,Makkalchon Korean Restaurant 맛깔촌,Korean Restaurant,43.744945,-79.296494
3,581cad6a7c74e15859a6f890,Scaddabush Italian Kitchen & Bar,Italian Restaurant,43.644737,-79.385355
4,4ae7b27df964a52068ad21e3,Japango,Sushi Restaurant,43.655268,-79.385165


### 3. Analyzing the data

#### Sorting restaurants by rating

In [13]:
def SortRestaurants(best_restaurants):
    
    ratings = list()
    prices = list()
    
    for index, restaurant in best_restaurants.iterrows():
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            restaurant["id"],
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
        # make the GET request
        details = requests.get(url).json()
        try:
            ratings.append(details["response"]["venue"]["rating"])
        except:
            ratings.append(5)
        try:
            prices.append(details["response"]["venue"]["price"]["tier"])
        except:
            prices.append(3)
        
    
    best_restaurants["price"] = prices
    best_restaurants["rating"] = ratings
    return(best_restaurants.sort_values(by = 'rating', ascending = False).reset_index(drop=True).copy())

In [14]:
restaurants_sorted = SortRestaurants(best_restaurants)
restaurants_sorted.head(20)

,id,name,categories,lat,lng,price,rating
0,4ad4c05ef964a520bff620e3,The Distillery Historic District,Historic Site,43.650244,-79.359323,3,9.3
1,4ee8f32602d5895bd7dce1b1,Gusto 101,Italian Restaurant,43.644988,-79.400270,2,9.0
2,529612de11d2ab526191ccc9,Pai,Thai Restaurant,43.647923,-79.388579,2,9.0
3,4ad4c05ef964a52096f620e3,CN Tower,Monument / Landmark,43.642536,-79.387182,3,8.9
4,4adb969ef964a520332921e3,Banjara Indian Cuisine,Indian Restaurant,43.662916,-79.421911,1,8.6
5,4b15383bf964a52079a923e3,Capocaccia Café,Italian Restaurant,43.685915,-79.393305,3,8.5
6,4b49183ff964a520a46526e3,Terroni,Italian Restaurant,43.650927,-79.375602,3,8.5
7,4ae86fb7f964a52053af21e3,Balsamico,Italian Restaurant,43.701505,-79.397162,2,8.5
8,4ae338e1f964a520e39121e3,Le Sélect Bistro,French Restaurant,43.643699,-79.396667,2,8.4
9,4aec5a49f964a52044c621e3,Tutto Pronto,Italian Restaurant,43.728235,-79.418086,2,8.4


#### Best restaurants in Toronto on a map

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, name, category in zip(restaurants_sorted['lat'], restaurants_sorted['lng'], restaurants_sorted['name'], restaurants_sorted['categories']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

#### Get the weights of the restaurants for a travel plan

In [16]:
weights = np.arange(1,0,-0.05)*np.arange(1,0,-0.05)*np.arange(1,0,-0.05)
    
print(weights)
print(weights.shape)

[1.00000e+00 8.57375e-01 7.29000e-01 6.14125e-01 5.12000e-01 4.21875e-01
 3.43000e-01 2.74625e-01 2.16000e-01 1.66375e-01 1.25000e-01 9.11250e-02
 6.40000e-02 4.28750e-02 2.70000e-02 1.56250e-02 8.00000e-03 3.37500e-03
 1.00000e-03 1.25000e-04]
(20,)


In [17]:
restaurants_sorted["weights"] = weights

In [18]:
def getNearbyVenues(names, latitudes, longitudes, query, radius=500, LIMIT = 40):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            query,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=TorontoDf['Neighborhood'],
                                   latitudes=TorontoDf['Latitude'],
                                   longitudes=TorontoDf['Longitude'],
                                   query = "restaurants")

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(868, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
1,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Regent Park, Harbourfront",43.65426,-79.360636,Brick Street Bakery,43.650574,-79.359539,Bakery
3,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


#### Let's group by Neighborhood and order by rating to see the neighborhoods with the best restaurants

In [21]:
best_neigh = pd.DataFrame(columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Rating'])

best_neigh["Neighborhood"] = toronto_venues["Neighborhood"].unique()
best_neigh["Rating"] = np.zeros(len(best_neigh["Neighborhood"]))

for indexN, neigh in best_neigh.iterrows():
    for indexV, venue in (toronto_venues[toronto_venues["Neighborhood"] == neigh["Neighborhood"]]).iterrows():
        best_neigh.iat[indexN,1] = venue["Neighborhood Latitude"]
        best_neigh.iat[indexN,2] = venue["Neighborhood Longitude"]
        r = restaurants_sorted[restaurants_sorted["name"] == venue["Venue"]]
        if r.shape[0] > 0:
            best_neigh.iat[indexN,3] += 1*r.iat[0,6]


In [22]:
top5 = best_neigh.sort_values(by='Rating', ascending = False).reset_index(drop=True).head(5).copy()

In [23]:
top5

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Rating
0,"Richmond, Adelaide, King",43.6506,-79.3846,17.3
1,"Commerce Court, Victoria Hotel",43.6482,-79.3798,16.8
2,Central Bay Street,43.658,-79.3874,16.1
3,St. James Town,43.6515,-79.3754,8.5
4,"First Canadian Place, Underground city",43.6484,-79.3823,8.3


#### Showing 5 best neighborhoods

In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

num = 1
# add markers to map
for lat, lng, name in zip(top5['Neighborhood Latitude'], top5['Neighborhood Longitude'], top5['Neighborhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    folium.map.Marker(
    [lat+0.0015, lng-0.0007],
    icon=DivIcon(
        icon_size=(75,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt">%s</div>' % num,
        )
    ).add_to(map_toronto)
    num+=1
    
map_toronto

#### Calling Foursquare to get the best hotels in the best neighborhood

In [25]:
LIMIT = 3 # limit of number of venues returned by Foursquare API

query = "Good Hotels"

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    top5.at[0,"Neighborhood Latitude"], 
    top5.at[0,"Neighborhood Longitude"],
    query,
    radius, 
    LIMIT)

In [26]:
best_hotels = requests.get(url).json()

In [27]:
hotels = best_hotels['response']['groups'][0]['items']
    
best_hotels = json_normalize(hotels) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
best_hotels = best_hotels.loc[:, filtered_columns]

# filter the category for each row
best_hotels['venue.categories'] = best_hotels.apply(get_category_type, axis=1)

# clean columns
best_hotels.columns = [col.split(".")[-1] for col in best_hotels.columns]

best_hotels.head(3)

<ipython-input-27-a9c99ef3c08b>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  best_hotels = json_normalize(hotels) # flatten JSON


,id,name,categories,lat,lng
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,Hotel,43.650594,-79.384530
1,4e31b74252b131dcebb08743,Shangri-La Toronto,Hotel,43.649129,-79.386557
2,52ce14b0498e50457ce11780,DoubleTree by Hilton,Hotel,43.654608,-79.385942


#### Showing the map with the best hotels in those neighborhoods

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, name in zip(best_hotels['lat'], best_hotels['lng'], best_hotels['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto